# Dependency Graph - Complete Solutions

In [ ]:
from typing import List, Set, Dict, Optional, Tuple
from collections import defaultdict, deque
from dataclasses import dataclass

@dataclass
class NodeConfig:
    name: str
    depends_on: List[str]

class DependencyError(Exception):
    def __init__(self, message: str, cycle: Optional[List[str]] = None):
        super().__init__(message)
        self.cycle = cycle

## Complete DependencyGraph Implementation

In [ ]:
class DependencyGraph:
    """Complete dependency graph implementation with all features."""
    
    def __init__(self, nodes: List[NodeConfig]):
        self.nodes = {node.name: node for node in nodes}
        self.adjacency_list: Dict[str, List[str]] = defaultdict(list)
        self.reverse_adjacency_list: Dict[str, List[str]] = defaultdict(list)
        
        self._build_graph()
        self._validate_graph()
    
    def _build_graph(self) -> None:
        for node in self.nodes.values():
            for dependency in node.depends_on:
                self.adjacency_list[dependency].append(node.name)
                self.reverse_adjacency_list[node.name].append(dependency)
    
    def _validate_graph(self) -> None:
        self._check_missing_dependencies()
        self._check_cycles()
    
    def _check_missing_dependencies(self) -> None:
        missing_deps = []
        
        for node in self.nodes.values():
            for dependency in node.depends_on:
                if dependency not in self.nodes:
                    missing_deps.append((node.name, dependency))
        
        if missing_deps:
            errors = [
                f"Node '{node}' depends on '{dep}' which doesn't exist"
                for node, dep in missing_deps
            ]
            raise DependencyError("Missing dependencies found:\n  " + "\n  ".join(errors))
    
    def _check_cycles(self) -> None:
        visited = set()
        rec_stack = set()
        
        def visit(node: str, path: List[str]) -> Optional[List[str]]:
            if node in rec_stack:
                cycle_start = path.index(node)
                return path[cycle_start:] + [node]
            
            if node in visited:
                return None
            
            visited.add(node)
            rec_stack.add(node)
            path.append(node)
            
            for dependent in self.adjacency_list[node]:
                cycle = visit(dependent, path[:])
                if cycle:
                    return cycle
            
            rec_stack.remove(node)
            return None
        
        for node_name in self.nodes.keys():
            if node_name not in visited:
                cycle = visit(node_name, [])
                if cycle:
                    raise DependencyError("Circular dependency detected", cycle=cycle)
    
    def topological_sort(self) -> List[str]:
        in_degree = {name: 0 for name in self.nodes.keys()}
        for node in self.nodes.values():
            for dependency in node.depends_on:
                in_degree[node.name] += 1
        
        queue = deque([name for name, degree in in_degree.items() if degree == 0])
        sorted_nodes = []
        
        while queue:
            node_name = queue.popleft()
            sorted_nodes.append(node_name)
            
            for dependent in self.adjacency_list[node_name]:
                in_degree[dependent] -= 1
                if in_degree[dependent] == 0:
                    queue.append(dependent)
        
        if len(sorted_nodes) != len(self.nodes):
            raise DependencyError("Failed to create topological sort (likely cycle)")
        
        return sorted_nodes
    
    def get_execution_layers(self) -> List[List[str]]:
        in_degree = {name: len(node.depends_on) for name, node in self.nodes.items()}
        
        layers = []
        remaining = set(self.nodes.keys())
        
        while remaining:
            current_layer = [name for name in remaining if in_degree[name] == 0]
            
            if not current_layer:
                raise DependencyError("Cannot create execution layers (likely cycle)")
            
            layers.append(current_layer)
            
            for node_name in current_layer:
                remaining.remove(node_name)
                for dependent in self.adjacency_list[node_name]:
                    if dependent in remaining:
                        in_degree[dependent] -= 1
        
        return layers
    
    def get_dependencies(self, node_name: str) -> Set[str]:
        if node_name not in self.nodes:
            raise ValueError(f"Node '{node_name}' not found")
        
        dependencies = set()
        queue = deque([node_name])
        
        while queue:
            current = queue.popleft()
            for dependency in self.reverse_adjacency_list[current]:
                if dependency not in dependencies:
                    dependencies.add(dependency)
                    queue.append(dependency)
        
        return dependencies
    
    def get_dependents(self, node_name: str) -> Set[str]:
        if node_name not in self.nodes:
            raise ValueError(f"Node '{node_name}' not found")
        
        dependents = set()
        queue = deque([node_name])
        
        while queue:
            current = queue.popleft()
            for dependent in self.adjacency_list[current]:
                if dependent not in dependents:
                    dependents.add(dependent)
                    queue.append(dependent)
        
        return dependents
    
    def get_independent_nodes(self) -> List[str]:
        return [name for name, node in self.nodes.items() if not node.depends_on]
    
    def visualize(self) -> str:
        lines = ["Dependency Graph:", ""]
        
        layers = self.get_execution_layers()
        for i, layer in enumerate(layers):
            lines.append(f"Layer {i + 1}:")
            for node_name in sorted(layer):
                node = self.nodes[node_name]
                deps = (
                    f" (depends on: {', '.join(sorted(node.depends_on))})"
                    if node.depends_on
                    else ""
                )
                lines.append(f"  - {node_name}{deps}")
            lines.append("")
        
        return "\n".join(lines)

## Challenge Solutions

### Challenge 1: Critical Path

In [ ]:
def get_critical_path(self) -> Tuple[List[str], int]:
    """Find the longest path through the graph.
    
    Returns:
        Tuple of (path, length)
    """
    # Calculate longest path to each node
    longest_path = {name: (0, []) for name in self.nodes.keys()}
    
    # Process nodes in topological order
    for node_name in self.topological_sort():
        current_length, current_path = longest_path[node_name]
        
        # Update dependents
        for dependent in self.adjacency_list[node_name]:
            new_length = current_length + 1
            if new_length > longest_path[dependent][0]:
                longest_path[dependent] = (new_length, current_path + [node_name])
    
    # Find node with longest path
    max_node = max(longest_path.items(), key=lambda x: x[1][0])
    path = max_node[1][1] + [max_node[0]]
    length = max_node[1][0]
    
    return path, length

DependencyGraph.get_critical_path = get_critical_path

In [ ]:
# Test critical path
nodes = [
    NodeConfig('A', []),
    NodeConfig('B', ['A']),
    NodeConfig('C', ['A']),
    NodeConfig('D', ['B']),
    NodeConfig('E', ['C']),
    NodeConfig('F', ['D', 'E'])
]

graph = DependencyGraph(nodes)
path, length = graph.get_critical_path()

print(f"Critical path: {' → '.join(path)}")
print(f"Length: {length} steps")
print(f"Minimum execution time: {length + 1} time units (assuming 1 unit per node)")

### Challenge 2: Graph Diff

In [ ]:
def diff(self, other: 'DependencyGraph') -> Dict[str, Set[str]]:
    """Compare with another graph.
    
    Returns:
        Dict with keys: 'added', 'removed', 'modified'
    """
    old_names = set(self.nodes.keys())
    new_names = set(other.nodes.keys())
    
    added = new_names - old_names
    removed = old_names - new_names
    common = old_names & new_names
    
    # Check for modified dependencies
    modified = set()
    for name in common:
        old_deps = set(self.nodes[name].depends_on)
        new_deps = set(other.nodes[name].depends_on)
        if old_deps != new_deps:
            modified.add(name)
    
    return {
        'added': added,
        'removed': removed,
        'modified': modified
    }

DependencyGraph.diff = diff

In [ ]:
# Test graph diff
old_nodes = [
    NodeConfig('A', []),
    NodeConfig('B', ['A']),
    NodeConfig('C', ['A'])
]

new_nodes = [
    NodeConfig('A', []),
    NodeConfig('B', ['A', 'D']),  # Modified
    NodeConfig('D', [])  # Added
    # C removed
]

old_graph = DependencyGraph(old_nodes)
new_graph = DependencyGraph(new_nodes)

changes = old_graph.diff(new_graph)
print("Changes:")
for change_type, nodes in changes.items():
    if nodes:
        print(f"  {change_type}: {sorted(nodes)}")

### Challenge 3: Minimal Rebuild Set

In [ ]:
def get_rebuild_set(self, changed_nodes: List[str]) -> Set[str]:
    """Get all nodes that need re-execution when given nodes change.
    
    Includes the changed nodes themselves plus all transitive dependents.
    """
    rebuild_set = set(changed_nodes)
    
    for node in changed_nodes:
        if node in self.nodes:
            rebuild_set.update(self.get_dependents(node))
    
    return rebuild_set

DependencyGraph.get_rebuild_set = get_rebuild_set

In [ ]:
# Test rebuild set
pipeline = [
    NodeConfig('extract', []),
    NodeConfig('clean', ['extract']),
    NodeConfig('validate', ['extract']),
    NodeConfig('aggregate', ['clean']),
    NodeConfig('report', ['aggregate', 'validate'])
]

graph = DependencyGraph(pipeline)

# If 'extract' changes, what needs to rebuild?
rebuild = graph.get_rebuild_set(['extract'])
print(f"If 'extract' changes, rebuild: {sorted(rebuild)}")

# If 'clean' changes?
rebuild = graph.get_rebuild_set(['clean'])
print(f"If 'clean' changes, rebuild: {sorted(rebuild)}")

## Bonus: Graph Metrics

In [ ]:
def get_metrics(self) -> Dict[str, any]:
    """Calculate graph metrics."""
    layers = self.get_execution_layers()
    critical_path, critical_length = self.get_critical_path()
    
    return {
        'total_nodes': len(self.nodes),
        'total_edges': sum(len(deps) for deps in self.adjacency_list.values()),
        'layers': len(layers),
        'max_parallelism': max(len(layer) for layer in layers),
        'critical_path_length': critical_length,
        'independent_nodes': len(self.get_independent_nodes()),
        'avg_dependencies': sum(len(n.depends_on) for n in self.nodes.values()) / len(self.nodes)
    }

DependencyGraph.get_metrics = get_metrics

In [ ]:
# Comprehensive example
complex_pipeline = [
    NodeConfig('extract_users', []),
    NodeConfig('extract_orders', []),
    NodeConfig('extract_products', []),
    NodeConfig('clean_users', ['extract_users']),
    NodeConfig('clean_orders', ['extract_orders']),
    NodeConfig('clean_products', ['extract_products']),
    NodeConfig('validate_users', ['clean_users']),
    NodeConfig('validate_orders', ['clean_orders']),
    NodeConfig('enrich_orders', ['validate_orders', 'clean_products']),
    NodeConfig('user_metrics', ['validate_users', 'enrich_orders']),
    NodeConfig('product_metrics', ['enrich_orders']),
    NodeConfig('dashboard', ['user_metrics', 'product_metrics'])
]

graph = DependencyGraph(complex_pipeline)

print("Graph Metrics:")
print("=" * 50)
for key, value in graph.get_metrics().items():
    print(f"{key:25s}: {value}")

print("\n" + graph.visualize())

## 🎓 Key Insights

### Algorithm Choices

1. **Kahn's vs DFS for topological sort**
   - Kahn's: Easier to understand, natural for execution layers
   - DFS: More elegant, better for finding all possible orders

2. **BFS for dependency queries**
   - Level-by-level exploration
   - Easy to track visited nodes
   - Could use DFS, but BFS more intuitive

3. **Two adjacency lists**
   - Memory overhead but O(1) access in both directions
   - Critical for performance with large graphs

### Performance Considerations

- All operations: O(N + E) where N = nodes, E = edges
- Space: O(N + E) for adjacency lists
- Validation happens once in constructor

### Real-World Applications

1. **Build Systems**: Make, Bazel, Gradle
2. **Data Pipelines**: Airflow, Dagster, Prefect
3. **Package Managers**: npm, pip, cargo
4. **CI/CD**: GitHub Actions, CircleCI
5. **Spreadsheets**: Cell dependency tracking